<a href="https://colab.research.google.com/github/yadavayush834/emotion-classifier/blob/main/progress_tracker2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U transformers accelerate peft trl bitsandbytes datasets

from huggingface_hub import login
import torch

# Paste your Hugging Face token here when prompted
login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Load the emotion dataset (split into train/validation/test)
dataset = load_dataset("dair-ai/emotion")

# Print some info to check
print(dataset)
print(dataset['train'][0])  # Example entry

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
{'text': 'i didnt feel humiliated', 'label': 0}


In [ ]:
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from transformers import TrainingArguments
import numpy as np

# Emotion label mapping (for reference)
id2label = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
label2id = {v: k for k, v in id2label.items()}

# Function to format examples as instructions
def formatting_prompts_func(examples):
    texts = examples["text"]
    labels = examples["label"]
    formatted = []
    for text, label in zip(texts, labels):
        prompt = f"Analyze the emotion in this text and respond with only one word: sadness, joy, love, anger, fear, or surprise.\n\nText: {text}\n\nEmotion:"
        target = id2label[label]
        formatted.append(prompt + " " + target)
    return {"text": formatted}

# Apply formatting to dataset
dataset_formatted = dataset.map(formatting_prompts_func, batched=True)

print("Dataset formatted and ready!")
print("Example formatted entry:")
print(dataset_formatted["train"][0]["text"])

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset formatted and ready!
Example formatted entry:
Analyze the emotion in this text and respond with only one word: sadness, joy, love, anger, fear, or surprise.

Text: i didnt feel humiliated

Emotion: sadness


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# 4-bit quantization config (updated compute_dtype to float16)
quantization_config = BitsAndBytesConfig(
    #load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model_id = "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16,  # Ensure model is loaded in float16
)

# Prepare model for QLoRA
model = prepare_model_for_kbit_training(model)

# LoRA config (efficient, low memory)
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

print("Model loaded and PEFT applied with float16 dtype!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded and PEFT applied with float16 dtype!


In [ ]:
import os
os.environ["ACCELERATE_MIXED_PRECISION"] = "no"

from trl import SFTTrainer, SFTConfig

# Limit dataset size
train_dataset = dataset_formatted["train"].select(range(8000))
eval_dataset = dataset_formatted["validation"]

# SFTConfig with fp16
sft_config = SFTConfig(
    output_dir="./gemma2_emotion_finetuned",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    fp16=False, # Temporarily set to False for diagnosis
    bf16=False,  # Explicitly disable bf16
    logging_steps=50,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=200,
    warmup_steps=100,
    report_to="none",
    load_best_model_at_end=False,
    max_length=512,
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("Starting training with fp16 fix... This will take 20-40 minutes. Keep the tab open!")
trainer.train()
print("Training complete! 🎉")

Starting training with fp16 fix... This will take 20-40 minutes. Keep the tab open!


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
200,1.463800,1.406642,2.010964,350097.000000,0.755709


Training complete! 🎉


In [ ]:
# Test the fine-tuned model

def predict_emotion(text):
    prompt = f"Analyze the emotion in this text and respond with only one word: sadness, joy, love, anger, fear, or surprise.\n\nText: {text}\n\nEmotion:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False,  # Greedy decoding for consistent results
        temperature=0.0,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract just the emotion word after "Emotion:"
    emotion = response.split("Emotion:")[-1].strip().split()[0].lower()
    return emotion

# Test with some examples
test_texts = [
    "I just got promoted at work! I'm so excited!",
    "My dog passed away yesterday. I miss him so much.",
    "I can't believe he lied to me again. I'm furious!",
    "This movie is so beautiful, I'm in love with it.",
    "I'm really nervous about the exam tomorrow.",
    "Wow, that's unexpected! What a surprise."
]

print("Testing the fine-tuned model:\n")
for text in test_texts:
    emotion = predict_emotion(text)
    print(f"Text: {text}")
    print(f"Predicted Emotion: {emotion.capitalize()}\n")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in Gemma2DecoderLayer. Setting `past_key_values=None`.


Testing the fine-tuned model:



/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Text: I just got promoted at work! I'm so excited!
Predicted Emotion: Joy

Text: My dog passed away yesterday. I miss him so much.
Predicted Emotion: Sadness

Text: I can't believe he lied to me again. I'm furious!
Predicted Emotion: Anger

Text: This movie is so beautiful, I'm in love with it.
Predicted Emotion: Love

Text: I'm really nervous about the exam tomorrow.
Predicted Emotion: Fear

Text: Wow, that's unexpected! What a surprise.
Predicted Emotion: Surprise!!!!!!!!!



In [ ]:
# Your custom todo categories
custom_categories = [
    "work", "school", "homework", "physical exercise", "meditation",
    "relax", "personal growth", "family", "friends", "health", "shopping", "chores"
]

# Examples: feel free to add more later!
custom_data = [
    # work
    ("Finish the quarterly report for the team meeting", "work"),
    ("Reply to client emails before 5pm", "work"),
    ("Prepare presentation slides for tomorrow", "work"),
    ("Call the supplier about the delayed order", "work"),

    # school
    ("Study chapter 5 for math test", "school"),
    ("Read pages 100-150 of history book", "school"),
    ("Complete physics lab report", "school"),

    # homework
    ("Do English essay on Shakespeare", "homework"),
    ("Solve 20 algebra problems", "homework"),
    ("Finish science project poster", "homework"),

    # physical exercise
    ("Go for a 5km run in the park", "physical exercise"),
    ("Do 30 minutes of yoga", "physical exercise"),
    ("Gym workout - chest and arms", "physical exercise"),
    ("Play badminton with friends", "physical exercise"),

    # meditation / relax
    ("Meditate for 15 minutes in the morning", "meditation"),
    ("Do breathing exercises before bed", "meditation"),
    ("Listen to calming music and relax", "relax"),
    ("Take a hot bath and read a book", "relax"),
    ("Watch a light comedy show", "relax"),

    # others
    ("Call mom and catch up", "family"),
    ("Buy groceries after work", "shopping"),
    ("Clean the kitchen and do laundry", "chores"),
    ("Journal about my goals", "personal growth"),
]

print(f"Created {len(custom_data)} custom examples with {len(custom_categories)} categories!")

Created 23 custom examples with 12 categories!


In [ ]:
# --- Your existing categories ---
custom_categories = [
    "work", "school", "homework", "physical exercise", "meditation",
    "relax", "personal growth", "family", "friends", "health", "shopping", "chores"
]

# --- Additional Data to balance the dataset ---
additional_data = [
    # --- WORK (Business, administrative, technical) ---
    ("Schedule the quarterly performance review", "work"),
    ("Debug the API connection issue", "work"),
    ("Update the Jira tickets for the sprint", "work"),
    ("Draft the proposal for the new marketing campaign", "work"),
    ("Fill out the expense report for the business trip", "work"),
    ("Attend the HR training session on compliance", "work"),

    # --- SCHOOL (Attendance, studying, administrative) ---
    ("Register for next semester's classes", "school"),
    ("Meet with the academic advisor", "school"),
    ("Attend the guest lecture in the auditorium", "school"),
    ("Review flashcards for the biology final", "school"),
    ("Pay the tuition fees online", "school"),

    # --- HOMEWORK (Submission, writing, solving) ---
    ("Write the abstract for the research paper", "homework"),
    ("Submit the coding assignment to Github", "homework"),
    ("Translate the French paragraph for tomorrow", "homework"),
    ("Finish the group project slides", "homework"),
    ("Complete the online quiz for Chapter 4", "homework"),

    # --- PHYSICAL EXERCISE (Sports, gym, cardio) ---
    ("Attend the spin class at 6 PM", "physical exercise"),
    ("Leg day: Squats and lunges", "physical exercise"),
    ("Swim 20 laps at the community pool", "physical exercise"),
    ("Stretch for 15 minutes", "physical exercise"),
    ("Go for a hike on the nature trail", "physical exercise"),

    # --- MEDITATION (Mindfulness, breathing) ---
    ("Practice mindfulness using the Headspace app", "meditation"),
    ("Do a body scan meditation", "meditation"),
    ("Spend 10 minutes in silence", "meditation"),
    ("Focus on deep breathing techniques", "meditation"),

    # --- RELAX (Leisure, passive entertainment) ---
    ("Play video games for an hour", "relax"),
    ("Binge-watch the new series on Netflix", "relax"),
    ("Sit on the balcony and drink tea", "relax"),
    ("Nap for 20 minutes", "relax"),
    ("Scroll through social media", "relax"),

    # --- PERSONAL GROWTH (Skill building, reading, hobbies) ---
    ("Practice Spanish vocabulary on Duolingo", "personal growth"),
    ("Read 'Atomic Habits' for 30 minutes", "personal growth"),
    ("Watch a tutorial on Python programming", "personal growth"),
    ("Practice guitar chords", "personal growth"),
    ("Listen to a podcast about investing", "personal growth"),

    # --- FAMILY (Helping, events, communication) ---
    ("Drive dad to his eye appointment", "family"),
    ("Plan the menu for sister's birthday", "family"),
    ("Video call with grandma", "family"),
    ("Pick up little brother from soccer practice", "family"),
    ("Help mom fix the printer", "family"),

    # --- FRIENDS (Socializing, planning) ---
    ("Grab coffee with Sarah downtown", "friends"),
    ("Text the group chat about the weekend trip", "friends"),
    ("Buy a housewarming gift for Mike", "friends"),
    ("Go to the movies with the guys", "friends"),
    ("RSVP to Jessica's wedding", "friends"),

    # --- HEALTH (Medical, self-care, appointments) ---
    ("Book a dentist appointment for cleaning", "health"),
    ("Refill prescription at the pharmacy", "health"),
    ("Drink 2 liters of water today", "health"),
    ("Take daily vitamins", "health"),
    ("Schedule annual physical exam", "health"),

    # --- SHOPPING (Purchasing goods) ---
    ("Buy new running shoes", "shopping"),
    ("Order toner for the printer", "shopping"),
    ("Pick up milk and eggs on the way home", "shopping"),
    ("Browse for a new winter coat", "shopping"),
    ("Return the defective headphones", "shopping"),

    # --- CHORES (Cleaning, maintenance) ---
    ("Vacuum the living room rug", "chores"),
    ("Load and start the dishwasher", "chores"),
    ("Water the indoor plants", "chores"),
    ("Take the car to the car wash", "chores"),
    ("Fold the laundry", "chores"),
    ("Change the lightbulb in the hallway", "chores"),
]

# Merge the lists
custom_data.extend(additional_data)

print(f"Total dataset size: {len(custom_data)} examples across {len(custom_categories)} categories.")

Total dataset size: 84 examples across 12 categories.


In [ ]:
from datasets import Dataset, concatenate_datasets, Features, Value

# Create custom dataset
custom_texts = [text for text, label in custom_data]
custom_labels = [label for text, label in custom_data]

custom_dataset = Dataset.from_dict({
    "text": custom_texts,
    "label": custom_labels  # Now labels are strings like "work", not numbers
})

# Update formatting function to handle string labels (custom categories)
def formatting_prompts_func_multi(examples):
    texts = examples["text"]
    labels = examples["label"]
    formatted = []

    # Combine all possible outputs from both emotion and custom datasets
    all_possible_outputs = custom_categories + list(id2label.values())
    possible_outputs_str = ", ".join(sorted(list(set(all_possible_outputs)))) # Use set and sort for unique and ordered list

    for text, label in zip(texts, labels):
        if isinstance(label, int):
            # If label is an integer, it's from the emotion dataset, use id2label
            target_label_str = id2label[label]
        else:
            # If label is already a string, it's from the custom dataset
            target_label_str = label

        prompt = f"Analyze this todo task and respond with ONLY one category from: {possible_outputs_str}\n\nTask: {text}\n\nCategory:"
        formatted.append(prompt + " " + target_label_str)
    return {"text": formatted}

# Define the desired output features
output_features = Features({
    'text': Value('string')
})

# Format both datasets, explicitly setting the output features and removing original columns
emotion_formatted = dataset["train"].select(range(8000)).map(
    formatting_prompts_func_multi,
    batched=True,
    features=output_features,
    remove_columns=dataset["train"].column_names # Explicitly remove original columns
)
custom_formatted = custom_dataset.map(
    formatting_prompts_func_multi,
    batched=True,
    features=output_features,
    remove_columns=custom_dataset.column_names # Explicitly remove original columns
)

# Combine them
combined_train = concatenate_datasets([emotion_formatted, custom_formatted])
combined_eval = dataset["validation"].map(
    formatting_prompts_func_multi,
    batched=True,
    features=output_features,
    remove_columns=dataset["validation"].column_names # Explicitly remove original columns
)

print(f"Combined dataset: {len(combined_train)} training examples")
print("Example:")
print(combined_train[0]["text"])
print("\nCustom example:")
print(combined_train[-1]["text"])  # Last one should be custom


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Combined dataset: 8084 training examples
Example:
Analyze this todo task and respond with ONLY one category from: anger, chores, family, fear, friends, health, homework, joy, love, meditation, personal growth, physical exercise, relax, sadness, school, shopping, surprise, work

Task: i didnt feel humiliated

Category: sadness

Custom example:
Analyze this todo task and respond with ONLY one category from: anger, chores, family, fear, friends, health, homework, joy, love, meditation, personal growth, physical exercise, relax, sadness, school, shopping, surprise, work

Task: Change the lightbulb in the hallway

Category: chores


In [ ]:
from trl import SFTTrainer, SFTConfig

# Use the combined datasets we just created
train_dataset = combined_train
eval_dataset = combined_eval  # Validation from emotion dataset, formatted the same way

sft_config = SFTConfig(
    output_dir="./gemma2_todo_classifier_final",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    fp16=False,          # Keep this False – that's what worked for you
    bf16=False,
    logging_steps=50,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=200,
    warmup_steps=100,
    report_to="none",
    load_best_model_at_end=False,
    max_length=512,
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("Starting FINAL training (emotions + your todo categories)... 15-30 minutes. Keep the tab open!")
trainer.train()
print("FINAL Training complete! Your smart todo classifier is ready! 🚀")

Adding EOS to train dataset:   0%|          | 0/8084 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8084 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/8084 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Starting FINAL training (emotions + your todo categories)... 15-30 minutes. Keep the tab open!


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
200,1.686200,1.796029,2.437733,495999.000000,0.821859


FINAL Training complete! Your smart todo classifier is ready! 🚀


In [ ]:
# Updated prediction function for our new multi-category model

def predict_category(text):
    possible = "sadness, joy, love, anger, fear, surprise, work, school, homework, physical exercise, meditation, relax, personal growth, family, friends, health, shopping, chores"
    prompt = f"Analyze this todo task and respond with ONLY one category from: {possible}\n\nTask: {text}\n\nCategory:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=8,
        do_sample=False,  # Greedy for consistent results
        temperature=0.0,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the word after "Category:"
    try:
        category = response.split("Category:")[-1].strip().split()[0].lower()
    except:
        category = "unknown"
    return category.capitalize()

# Test with mixed examples
test_tasks = [
    "I just got promoted! I'm over the moon!",                  # should → Joy
    "My best friend betrayed me, I'm so hurt",                  # should → Sadness or Anger
    "Finish the client proposal before deadline",               # should → Work
    "Study for biology exam tomorrow",                          # should → School or Homework
    "Do 50 pushups and go for a jog",                           # should → Physical exercise
    "Meditate for 20 minutes after work",                       # should → Meditation
    "Clean the house and do laundry",                           # should → Chores
    "Call mom and dad tonight",                                 # should → Family
    "Buy milk, eggs, and bread from the store",                 # should → Shopping
    "Take a long bath and watch Netflix",                       # should → Relax
]

print("Testing your final model:\n")
for task in test_tasks:
    pred = predict_category(task)
    print(f"Task: {task}")
    print(f"→ Predicted: {pred}\n")

Testing your final model:



/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Task: I just got promoted! I'm over the moon!
→ Predicted: Joy

Task: My best friend betrayed me, I'm so hurt
→ Predicted: Sadness

Task: Finish the client proposal before deadline
→ Predicted: Work

Task: Study for biology exam tomorrow
→ Predicted: School

Task: Do 50 pushups and go for a jog
→ Predicted: Work

Task: Meditate for 20 minutes after work
→ Predicted: Meditation

Task: Clean the house and do laundry
→ Predicted: Chores

Task: Call mom and dad tonight
→ Predicted: Family

Task: Buy milk, eggs, and bread from the store
→ Predicted: Shopping

Task: Take a long bath and watch Netflix
→ Predicted: Relax



In [ ]:
# Save the fine-tuned model and tokenizer
save_path = "./my_todo_classifier_final"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved to {save_path}!")

Model saved to ./my_todo_classifier_final!


In [ ]:
from google.colab import files
import shutil
import os

# Zip the entire saved folder (makes it easier to download)
zip_name = "my_todo_classifier_final.zip"
shutil.make_archive(base_name="my_todo_classifier_final", format="zip", root_dir="my_todo_classifier_final")

print("Model zipped! Downloading now...")

# Download the zip file
files.download("my_todo_classifier_final.zip")

Model zipped! Downloading now...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from huggingface_hub import login, upload_folder, create_repo
from pathlib import Path

# Log in (paste your token when asked)
login()

# Your model folder (from earlier save)
model_path = "./my_todo_classifier_final"

# Your Hugging Face username (replace with yours!)
your_username = "ayush123c"  # ← CHANGE THIS! e.g.,

# Repo name you'll create
repo_name = "todo-emotion-classifier"

# Full repo ID
repo_id = f"{your_username}/{repo_name}"

print(f"Attempting to create/upload to https://huggingface.co/{repo_id}")

# Create the repository if it doesn't exist
create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)

# Upload the entire folder
upload_folder(
    folder_path=model_path,
    repo_id=repo_id,
    repo_type="model",
    commit_message="My fine-tuned Gemma-2-2b for todo + emotion classification"
)

print("Upload complete! ─")
print(f"View your model at: https://huggingface.co/{repo_id}")

Attempting to create/upload to https://huggingface.co/ayush123c/todo-emotion-classifier


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ier_final/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...fier_final/tokenizer.json:  73%|#######3  | 25.2MB / 34.4MB            

  ...adapter_model.safetensors:   1%|1         |  623kB / 51.1MB            

  ...r_final/training_args.bin:   1%|1         |  75.0B / 6.22kB            

Upload complete! ─
View your model at: https://huggingface.co/ayush123c/todo-emotion-classifier


In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Base Gemma model
base_model_name = "google/gemma-2-2b-it"

# Your adapter repo
adapter_repo = "ayush123c/todo-emotion-classifier"

print("Loading base model (this takes 3-5 minutes)...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16,  # Saves memory
)

print("Loading your fine-tuned adapter on top...")
model = PeftModel.from_pretrained(base_model, adapter_repo)

print("Merging adapter into base model (this takes 5-10 minutes and ~20-30GB RAM)...")
merged_model = model.merge_and_unload()

# Save tokenizer from base (it's the same)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Save merged model locally
merged_save_path = "./my_merged_todo_classifier"
merged_model.save_pretrained(merged_save_path)
tokenizer.save_pretrained(merged_save_path)

print(f"Merged model saved to {merged_save_path}!")

Loading base model (this takes 3-5 minutes)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading your fine-tuned adapter on top...


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/51.1M [00:00<?, ?B/s]

Merging adapter into base model (this takes 5-10 minutes and ~20-30GB RAM)...


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the locally merged model and tokenizer
merged_save_path = "./my_merged_todo_classifier"
local_tokenizer = AutoTokenizer.from_pretrained(
    merged_save_path
)
local_model = AutoModelForCausalLM.from_pretrained(
    merged_save_path,
    device_map="auto",
    torch_dtype=torch.float16
)

# Updated prediction function for our new multi-category model using the local model
def predict_category_local(text):
    possible = "sadness, joy, love, anger, fear, surprise, work, school, homework, physical exercise, meditation, relax, personal growth, family, friends, health, shopping, chores"
    prompt = f"Analyze this todo task and respond with ONLY one category from: {possible}\n\nTask: {text}\n\nCategory:"

    inputs = local_tokenizer(prompt, return_tensors="pt").to(local_model.device)

    outputs = local_model.generate(
        **inputs,
        max_new_tokens=8,
        do_sample=False,  # Greedy for consistent results
        temperature=0.0,
    )

    response = local_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the word after "Category:"
    try:
        category = response.split("Category:")[-1].strip().split()[0].lower()
    except:
        category = "unknown"
    return category.capitalize()

# Test with mixed examples using the local prediction function
test_tasks = [
    "I just got promoted! I'm over the moon!",                  # should → Joy
    "My best friend betrayed me, I'm so hurt",                  # should → Sadness or Anger
    "Finish the client proposal before deadline",               # should → Work
    "Study for biology exam tomorrow",                          # should → School or Homework
    "Do 50 pushups and go for a jog",                           # should → Physical exercise
    "Meditate for 20 minutes after work",                       # should → Meditation
    "Clean the house and do laundry",                           # should → Chores
    "Call mom and dad tonight",                                 # should → Family
    "Buy milk, eggs, and bread from the store",                 # should → Shopping
    "Take a long bath and watch Netflix",                       # should → Relax
]

print("Testing your final model (local inference):\n")
for task in test_tasks:
    pred = predict_category_local(task)
    print(f"Task: {task}")
    print(f" → Predicted: {pred}\n")

TypeError: not a string